# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load in the csv data files
weather_df_path = "data/weatherdata.csv"

# Read the weather data 
weather_df = pd.read_csv(weather_df_path)

In [3]:
weather_df.head(5)

,City_name,lat,long,max_temp,humidity,cloud,wind_speed,country
0,bara,10.37,10.73,26.61,70,14,4.76,NG
1,boa vista,2.82,-60.67,29.00,66,20,2.10,BR
2,albany,42.60,-73.97,26.11,65,78,2.24,US
3,hobart,-42.88,147.33,8.33,65,75,6.70,AU
4,rikitea,-23.12,-134.97,21.42,70,0,9.43,PF


In [4]:
weather_df.dtypes

City_name      object
lat           float64
long          float64
max_temp      float64
humidity        int64
cloud           int64
wind_speed    float64
country        object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [57]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [58]:
#Set the city latitudes and longitudes
city_locations = weather_df[["lat", "long"]]

In [59]:
#Add the humidity "weight"
city_humidity = weather_df["humidity"]

In [60]:
#Create a Google heatmap
city_fig = gmaps.figure()

#Create a heat layer to the heatmap
heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

city_fig.add_layer(heat_layer)

city_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

My ideal holiday would be:
* Maximum of 32(C) but no cooler than 27(C)
* Wind less than 5kph
* Cloudiness at less than 15%
* Humidity under 65%

In [9]:
#Create a dataframe with cities with a temperature between 27 and 32 degree celsius
holiday_weather = weather_df.loc[(weather_df["max_temp"] >= 27) & (weather_df["max_temp"] <= 32)]

In [10]:
#Filter the dataframe to exclude cities with wind less than 5kph
holiday_weather  = holiday_weather.loc[holiday_weather["wind_speed"] < 5]

In [11]:
#Filter the dataframe to exclude cities with cloudiness under 10%
holiday_weather  = holiday_weather.loc[holiday_weather["cloud"] < 10]

In [12]:
#Filter the dataframe to exclude cities with humidity under 65%
holiday_weather  = holiday_weather.loc[holiday_weather["humidity"] < 50]

In [13]:
holiday_weather.head(15)

,City_name,lat,long,max_temp,humidity,cloud,wind_speed,country
47,laurel,39.10,-76.85,31.11,42,1,1.70,US
210,riyadh,24.69,46.72,32.00,29,0,3.10,SA
304,mecca,21.43,39.83,30.65,13,0,2.07,SA
405,cortez,37.35,-108.59,31.00,15,1,3.60,US
413,bud,39.45,-86.18,30.00,42,1,3.60,US
471,hit,33.64,42.83,29.85,18,0,2.72,IQ
515,saint paul,44.94,-93.09,31.67,48,1,3.60,US
555,weiser,44.25,-116.97,27.00,22,1,2.60,US
581,sioux lookout,50.10,-91.92,30.00,37,5,2.60,CA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
#Create a copy of the hotel weather df for use below
hotel_df = holiday_weather.copy()

In [33]:
#Reindex the new DF
hotel_df.reset_index()

,index,City_name,lat,long,max_temp,humidity,cloud,wind_speed,country
0,47,laurel,39.10,-76.85,31.11,42,1,1.70,US
1,210,riyadh,24.69,46.72,32.00,29,0,3.10,SA
2,304,mecca,21.43,39.83,30.65,13,0,2.07,SA
3,405,cortez,37.35,-108.59,31.00,15,1,3.60,US
4,413,bud,39.45,-86.18,30.00,42,1,3.60,US
5,471,hit,33.64,42.83,29.85,18,0,2.72,IQ
6,515,saint paul,44.94,-93.09,31.67,48,1,3.60,US
7,555,weiser,44.25,-116.97,27.00,22,1,2.60,US
8,581,sioux lookout,50.10,-91.92,30.00,37,5,2.60,CA


In [51]:
#Set the list to hold the response information
hotel = []

#Setup further parameters to put into the params dictionary below
#https://developers.google.com/places/web-service/supported_types
target_radius = 5000
target_type = "lodging"

#Loop through the city names to get the nearest hotel from the Google Maps API
for index, row in hotel_df.iterrows():
    
    #Get the target latitude and longitudes for putting into the parameters below
   # hotel_lat = hotel_df.loc[city]["lat"]
   # hotel_long = hotel_df.loc[city]["long"]
    target_coordinates = f"{row['lat']},{row['long']}"
    
    #Create the parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    #Set the base url
    url_hotel = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Create search query, make request and store in json
    hotel_request = requests.get(url_hotel, params=params)
    hotel_response = hotel_request.json()
    
    #Obtain the information as required / put in a message if unable to locate
    try:
        hotel.append(hotel_response["results"][0]["name"])
    except:
        hotel.append("No hotel nearby")


In [54]:
print(hotel)

['Days Inn & Suites by Wyndham Laurel Near Fort Meade', 'InterContinental Riyadh', 'Hotel Fairmont Makkah Clock Royal Tower', 'Sand Canyon National 9 Inn', 'Madsion Scouts Camp', 'Aboody Alhachem', 'InterContinental Saint Paul Riverfront', 'The Colonial Motel', "Knobby's Fly-In Camps"]


In [56]:
hotel_df["Hotel"] = hotel
hotel_df.head()

,City_name,lat,long,max_temp,humidity,cloud,wind_speed,country,Hotel
47,laurel,39.10,-76.85,31.11,42,1,1.70,US,Days Inn & Suites by Wyndham Laurel Near Fort ...
210,riyadh,24.69,46.72,32.00,29,0,3.10,SA,InterContinental Riyadh
304,mecca,21.43,39.83,30.65,13,0,2.07,SA,Hotel Fairmont Makkah Clock Royal Tower
405,cortez,37.35,-108.59,31.00,15,1,3.60,US,Sand Canyon National 9 Inn
413,bud,39.45,-86.18,30.00,42,1,3.60,US,Madsion Scouts Camp


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

Add in the markers only

In [70]:
# Add marker layer ontop of heat map
#Create a heat layer to the heatmap 
fig = gmaps.figure()
heat_layer2 = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

layers2 = gmaps.marker_layer(locations)

fig.add_layer(heat_layer2)
fig.add_layer(layers2)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

Add in the hotel names on the markers

In [124]:
# Convert columns to lists
hotel_list = hotel_df["Hotel"].tolist()
hotel_list

['Days Inn & Suites by Wyndham Laurel Near Fort Meade',
 'InterContinental Riyadh',
 'Hotel Fairmont Makkah Clock Royal Tower',
 'Sand Canyon National 9 Inn',
 'Madsion Scouts Camp',
 'Aboody Alhachem',
 'InterContinental Saint Paul Riverfront',
 'The Colonial Motel',
 "Knobby's Fly-In Camps"]

In [125]:
# Add marker layer ontop of heat map
#Create a heat layer to the heatmap 
fig = gmaps.figure()
heat_layer2 = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

layers2 = gmaps.marker_layer(
    locations,
    info_box_content=([f"Hotel: {Hotel}" for Hotel in hotel_list])
)

fig.add_layer(heat_layer2)
fig.add_layer(layers2)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))